## Hive Hands On 

### `Run word count for two files which are there in hdfs pg4300 , pg5000 downloaded from gutenburg project `

In [2]:
! hadoop jar /usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-2.7.2.jar \
-file /root/python_mr/word_count/count_mapper.py -mapper /root/python_mr/word_count/count_mapper.py \
-file  /root/python_mr/word_count/count_reducer.py -reducer  /root/python_mr/word_count/count_reducer.py \
-input /user/ashu/pg5000.txt \
-output /user/ashu/pg5000

16/03/24 03:03:17 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
16/03/24 03:03:17 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
packageJobJar: [/root/python_mr/word_count/count_mapper.py, /root/python_mr/word_count/count_reducer.py, /tmp/hadoop-unjar1458932490324193186/] [] /tmp/streamjob942824628985982113.jar tmpDir=null
16/03/24 03:03:18 INFO client.RMProxy: Connecting to ResourceManager at /10.211.55.101:8032
16/03/24 03:03:19 INFO client.RMProxy: Connecting to ResourceManager at /10.211.55.101:8032
16/03/24 03:03:20 INFO mapred.FileInputFormat: Total input paths to process : 1
16/03/24 03:03:20 INFO mapreduce.JobSubmitter: number of splits:2
16/03/24 03:03:20 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1458650496725_0024
16/03/24 03:03:20 INFO impl.YarnClientImpl: Submitted application application_1458650496725_0024
16/03/24 03:03:20 

In [ ]:
! hadoop jar /usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-2.7.2.jar \
-file /root/python_mr/word_count/count_mapper.py -mapper /root/python_mr/word_count/count_mapper.py \
-file  /root/python_mr/word_count/count_reducer.py -reducer  /root/python_mr/word_count/count_reducer.py \
-input /user/ashu/pg4300.txt \
-output /user/ashu/pg4300

__Check if the data is there in the output,
remove the \_SUCCESS__

```
hdfs dfs -ls -h /user/ashu/pg4300
```

```
hdfs dfs -rm /user/ashu/pg4300/_SUCCESS
```

```
hdfs dfs -rm /user/ashu/pg5000/_SUCCESS
```

### `Create Table schema for both`

```Sql
CREATE TABLE pg4300 (word STRING, freq
  INT)  ROW FORMAT
  DELIMITED FIELDS TERMINATED BY '\t'
  STORED AS TEXTFILE;
```

```Sql
CREATE TABLE pg5000 (word STRING, freq
  INT)  ROW FORMAT
  DELIMITED FIELDS TERMINATED BY '\t'
  STORED AS TEXTFILE;
```

```Sql 
DESCRIBE pg4300;
```

### `Load dataset into Hive:`

```Sql
LOAD DATA INPATH '/user/ashu/pg4300/'
      INTO TABLE pg4300;
```

```Sql
SELECT * FROM pg4300 LIMIT 10;
```


```Sql
LOAD DATA INPATH '/user/ashu/pg5000/'
      INTO TABLE pg5000;
```

```Sql
SELECT * FROM pg5000 LIMIT 10;
```

### `Lets run more sqls`

```Sql
SELECT * FROM pg4300 WHERE freq > 100 SORT BY freq ASC LIMIT 10;
```

```Sql
SELECT * FROM pg5000 WHERE freq > 100 SORT BY freq ASC LIMIT 10;
```


### `Most common frequency`

```Sql
SELECT freq, COUNT(1) AS f2
    FROM pg4300 GROUP BY freq
    SORT BY f2 DESC LIMIT 10;
```

```Sql
SELECT freq, COUNT(1) AS f2
    FROM pg5000 GROUP BY freq
    SORT BY f2 DESC LIMIT 10;
```


### `When you want to know what happens under the hood!`

```Sql
EXPLAIN SELECT freq, COUNT(1) AS f2
    FROM pg4300 GROUP BY freq
    SORT BY f2 DESC LIMIT 10;
```

## Joins
A powerful feature of Hive is the ability to create queries that join tables together

```Sql
CREATE TABLE merged
    (word STRING, pg4300_f INT,
    pg5000_f INT);
```

```Sql
INSERT OVERWRITE TABLE merged
  SELECT s.word, s.freq, k.freq FROM
  pg4300 s JOIN pg5000 k ON
  (s.word = k.word)
```

### `Let's look at the meged Table `

```Sql
SELECT * FROM merged LIMIT 20;
```

```Sql
SELECT word,pg4300_f, pg5000_f,
    (pg4300_f + pg5000_f) AS ss
    FROM merged SORT BY ss DESC
    LIMIT 20;

```

### `Reference`
- https://cwiki.apache.org/confluence/display/Hive/LanguageManual+Select
- https://cwiki.apache.org/confluence/display/Hive/LanguageManual+Explain    